In [ ]:
import boto3
import sagemaker
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.sklearn.processing import SKLearnProcessor

# Define SageMaker session and role
sagemaker_session = sagemaker.Session()
sagemaker_role = "arn:aws:iam::123456789012:role/service-role/AmazonSageMaker-ExecutionRole-20220101T000000"

# Define inputs and outputs for the preprocessing job
input_data = ParameterString(name="InputData", default_value="s3://my-bucket/input-data/")
output_data = ParameterString(name="OutputData", default_value="s3://my-bucket/output-data/")

# Define the SKLearnProcessor for the preprocessing job
sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0",
    role=sagemaker_role,
    instance_type="ml.m5.large",
    instance_count=1,
)

# Define the processing step for the preprocessing job
step_process = ProcessingStep(
    name="PreprocessStep",
    processor=sklearn_processor,
    inputs=[
        sagemaker.processing.Input(
            source=input_data,
            destination="/opt/ml/processing/input",
            s3_data_type="S3Prefix",
            s3_input_mode="File",
        )
    ],
    outputs=[
        sagemaker.processing.Output(
            source="/opt/ml/processing/output",
            destination=output_data,
            s3_upload_mode="EndOfJob",
        )
    ],
    code="preprocessing_script.py",
)

# Define the pipeline
pipeline = Pipeline(
    name="MyPipeline",
    parameters=[input_data, output_data],
    steps=[step_process],
    sagemaker_session=sagemaker_session,
)

# Create and start the pipeline
pipeline.upsert(role_arn=sagemaker_role)
execution = pipeline.start()
